hw_8. Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

In [2]:
def L2(en1,en2):
    K,Lee = len(en1),0.0
    for i in range(K):
        Lee += (en1[i] - en2[i])**2
    Lee = np.sqrt(Lee)
    return Lee

In [3]:
def Knn_accuracy(X,y):
    dim_N,dim_M = X.shape
    coin = 0
    for i in range(dim_N):
        dist,mark = [],[0,0,0]
        for j in range(dim_N):
            dist.append(L2(X[i],X[j]))
        ind = np.argsort(dist)
        for j in range(1,6):
            ij = y[ind[j]]
            mark[ij] += 1
        yp = np.argmax(mark)
        if yp == y[i]: coin += 1
    res = coin / (1.0*dim_N)
    print(coin,dim_N)
    return res

In [4]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X = iris.data
X.shape

(150, 4)

In [5]:
# Для начала отмасштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]

In [6]:
# Найдем собственные векторы и собственные значения
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i[0])

Собственные значения в порядке убывания:
437.7746724797993
137.10457072021043
22.013531335697234
3.1072254642928705


In [8]:
# Устроим цикл с почередным выводом  из дела последних компонент, проверяя точность классификации метода, 
# обученного на упрощенных наборах. Сначала - полный набор
y = iris.target
print(Knn_accuracy(X,y))

145 150
0.9666666666666667


In [9]:
# Сформируем вектор весов из собственных векторов, соответствующих первым N-1 главным компонентам
for i in [4,3,2,1]:
    print(f'Учитываем первые {i} компоненты:')
    W = eig_pairs[0][1].reshape(4,1)
    for j in range(1,i):
        W = np.hstack((W, eig_pairs[j][1].reshape(4,1)))
    Z = X_.dot(W)
    print(Knn_accuracy(Z,y))

Учитываем первые 4 компоненты:
142 150
0.9466666666666667
Учитываем первые 3 компоненты:
143 150
0.9533333333333334
Учитываем первые 2 компоненты:
137 150
0.9133333333333333
Учитываем первые 1 компоненты:
139 150
0.9266666666666666


In [ ]:
# Постепенный отказ от малых компонент слабо уменьшант точность классификации, 
# да еще и немонотонно. Настолько главная компонента главенствует надо всеми